In [1]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from numpy import float32
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



In [2]:
users_df = pd.read_json("users.json")
users_df


,user_id,name,city,street,favourite_genres,premium_user
0,101,Maurycy Lejk,Wrocław,al. Powstańców Śląskich 04,"[folk, brill building pop, dance pop]",True
1,102,Julian Rohde,Kraków,al. Rataja 793,"[classic rock, dance pop, singer-songwriter]",False
2,103,Jerzy Żelazo,Kraków,pl. Orzechowa 509,"[pop rock, roots rock, italian adult pop]",False
3,104,Ada Mazanek,Warszawa,pl. Zacisze 78/30,"[italian adult pop, tropical, vocal jazz]",False
4,105,Ernest Kukieła,Warszawa,al. Orla 40,"[country rock, new wave, alternative metal]",False
...,...,...,...,...,...,...
949,1050,Anita Kik,Radom,al. Nowa 89,"[post-teen pop, brill building pop, hard rock]",False
950,1051,Jan Szarpak,Kraków,ulica Kasztanowa 01/66,"[folk rock, rock, album rock]",False
951,1052,Konstanty Tomaszkiewicz,Wrocław,pl. Zaułek 858,"[singer-songwriter, post-teen pop, modern rock]",True
952,1053,Nela Parzyszek,Kraków,ul. Piekarska 21/30,"[new wave, metal, motown]",False


In [3]:
### One hot encoding of the favourite_genres column
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

users_df = users_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(users_df.pop('favourite_genres')),
                index=users_df.index,
                columns=mlb.classes_))

users_df = users_df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(users_df.pop('city')),
                index=users_df.index,
                columns=mlb.classes_))

In [4]:
users_df.head(10)

,user_id,name,street,premium_user,adult standards,album rock,alternative metal,alternative rock,argentine rock,art rock,...,n,o,r,s,w,y,z,ó,ł,ń
0,101,Maurycy Lejk,al. Powstańców Śląskich 04,True,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,1,0
1,102,Julian Rohde,al. Rataja 793,False,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
2,103,Jerzy Żelazo,pl. Orzechowa 509,False,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
3,104,Ada Mazanek,pl. Zacisze 78/30,False,0,0,0,0,0,0,...,0,0,1,1,1,0,1,0,0,0
4,105,Ernest Kukieła,al. Orla 40,False,0,0,1,0,0,0,...,0,0,1,1,1,0,1,0,0,0
5,106,Maciej Kopyto,pl. Plażowa 74,False,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
6,107,Ewa Niklewicz,al. Kilińskiego 01/73,True,1,1,0,0,0,0,...,0,0,1,1,1,0,1,0,0,0
7,108,Bianka Piszcz,pl. Owocowa 141,False,0,0,0,0,1,0,...,1,1,0,0,0,0,1,0,0,1
8,109,Elżbieta Ciak,aleja Borowa 84,False,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
9,110,Mikołaj Kawulok,ul. Działkowa 93/34,False,0,0,0,0,0,1,...,1,1,0,0,0,0,1,0,0,1


In [4]:
assert users_df.isna().sum().sum() == 0

In [5]:
sessions_df = pd.read_json("sessions.json")

In [6]:
sessions_filtered = pd.DataFrame()
time_comparison_df = pd.DataFrame()
sessions_df.loc[:, 'timestamp'] = pd.to_datetime(sessions_df.loc[:, 'timestamp'])
sessions_df.sort_values(by=['timestamp'])
# sessions_df['next_timestamp'] = sessions_df['timestamp'].shift(-1)
# sessions_df.set_index('timestamp', inplace=True)
sessions_df

/tmp/ipykernel_19735/2395552958.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  sessions_df.loc[:, 'timestamp'] = pd.to_datetime(sessions_df.loc[:, 'timestamp'])


,timestamp,user_id,track_id,event_type,session_id
0,2022-04-08 13:51:17.000,101,4iQeCUQXp6oxdYYkERY0LN,PLAY,124
1,2022-04-08 13:55:45.440,101,2A3sgyp07d7ksgjyBHYSUt,PLAY,124
2,2022-04-08 13:58:20.293,101,2A3sgyp07d7ksgjyBHYSUt,LIKE,124
3,2022-04-08 13:58:36.413,101,,ADVERTISEMENT,124
4,2022-04-08 13:58:41.413,101,1hqrYSqvNc9x3BETX1cZhk,PLAY,124
...,...,...,...,...,...
484209,2023-03-08 18:36:32.078,1054,3PbJjso7KugjaEA8nkUPfk,SKIP,12909
484210,2023-03-08 18:36:32.078,1054,4xjTSci9xaqmpTagOYmJmB,PLAY,12909
484211,2023-03-08 18:38:50.771,1054,5Rhw4Yn50VR8MQkUUKnxC5,PLAY,12909
484212,2023-03-08 18:44:20.544,1054,35YArv7YwevvrButbI2bPL,PLAY,12909


In [10]:
def calculate_ads_time(df):
    ads_mask = df.loc[:,'event_type'] == "ADVERTISEMENT"
    ads_time = np.timedelta64(0)
    for _, action in df.loc[ads_mask].iterrows():
        difference = action.next_timestamp - action.timestamp
        if difference < np.timedelta64(0):
            pass
        ads_time += difference
    return ads_time

In [11]:
for user_id, user_actions in sessions_df.groupby('user_id'):
    user_bought_premium = False
    all_ads_time = np.timedelta64(0)
    for session_id, session in user_actions.groupby('session_id'):
        if user_bought_premium:
            break
        premium_bought_mask = session.loc[:,'event_type'] == "BUY_PREMIUM"
        bought_premium_in_session = premium_bought_mask.any()

        if bought_premium_in_session:
            session = session.loc[session.index <= premium_bought_mask.loc[premium_bought_mask].index[0]]
            user_bought_premium = True
        
        sessions_filtered = pd.concat([sessions_filtered, session])
        session['next_timestamp'] = session['timestamp'].shift(-1)
        session.sort_values(by=['timestamp'])

        all_ads_time += calculate_ads_time(session)
    
    # ads_time = np.sum((user_actions.loc[ads_mask]["next_timestamp"] - user_actions.loc[ads_mask].index)/ np.timedelta64(1, 's'))
    session_time =  (user_actions.iloc[[-1]].timestamp - user_actions.iloc[[0]].timestamp)/ np.timedelta64(1, 's')
    
    session_times_df = pd.DataFrame(
        {
            'session_time': session_time,
            'ads_time': all_ads_time, 
         },
            index=[user_id], 
    )
    # print(session_time)
    time_comparison_df = pd.concat([time_comparison_df, session_times_df])
    
# sessions_filtered.set_index(['timestamp'], inplace=True)
sessions_filtered
time_comparison_df


/tmp/ipykernel_19735/786131389.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session['next_timestamp'] = session['timestamp'].shift(-1)
/tmp/ipykernel_19735/786131389.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  session['next_timestamp'] = session['timestamp'].shift(-1)
/tmp/ipykernel_19735/786131389.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [ ]:
X = users_df.drop(["premium_user", "street", "name"], axis=1) 
y = users_df.loc[:, "premium_user"] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
mindist = NearestCentroid()
mindist.fit(X_train, y_train)
# mindist
# X_train = ss.transform(X_train)
y_hat = mindist.predict(X_test)
score = accuracy_score(y_test, y_hat)
score 